In [2]:
# gruneisenTofrequency.py by G Krenzer 

import matplotlib.pyplot as plt
from pymatgen.phonon.bandstructure import PhononBandStructureSymmLine
from pymatgen.core.lattice import Lattice
import yaml 
import json
import numpy as np
import pandas as pd
import statistics 

# input values
    
n_modes = 12
Tmin = 0
Tstep = 100
Tmax = 400 
labels = {"\\Gamma": [0.0, 0.0, 0.0], "A": [0.0, 0.0, 1/2], "L": [0.0, 1/2, 1/2], 
          "M": [0.0, 1/2, 0.0], "K": [-1/3, 2/3, 0.0], "H": [-1/3, 2/3, 1/2]}

with open('gruneisen.yaml', 'r') as file:
    """Loads gruneisen.yaml and turn it into a python dictionary"""
    gruneisen_dict = yaml.load(file, Loader = yaml.FullLoader)

def inclusive_range(start, stop, step):
    """Returns a range list with stop added at the end, regardless of step"""
    out_list = list(range(start,stop,step))
    out_list.append(stop)
    
    return out_list

def get_reciprocal_vectors():
    """Finds the reciprocal lattice vectors WITH a facotr of 2*pi from the POSCAR file"""
    POSCAR = pd.read_csv('0/POSCAR', skiprows = 2, nrows = 3, sep="\s+", names = ['a1', 'a2', 'a3'])
    vectors = []
    for i in range(3):
        coordinates = []
        for j in range(3):
            coordinates.append(POSCAR.iat[i, j])
        vectors.append(coordinates)
    reciprocal_vectors = Lattice(vectors).reciprocal_lattice

    return reciprocal_vectors

def get_V0():
    """Finds the equilibrium volume from the e-v.dat file of the phonopy-qha workflow"""
    """Assumes the equilibrium volume used with phonopy_qha is the one used for the phonopy-gruneisen calculation"""
    ev_np = np.loadtxt('e-v.dat')
    vol = []
    for ev in ev_np: 
        vol.append(ev[:][0])
    V0 = statistics.median(vol)
    
    return V0

def get_volumes_T():
    """Finds the volumes that correspond to the given temperatures"""
    vol_df = pd.read_csv('volume-temperature.dat', sep="\s+", names = ['Temperature', 'Volume'])
    volumes_t = []
    for T in inclusive_range(Tmin, Tmax, Tstep):
        index = vol_df.loc[vol_df['Temperature'] == T].index[0]
        vol_t = vol_df.at[index, 'Volume']
        volumes_t.append(vol_t)
        
    return volumes_t

def get_q():
    """Finds q-points"""
    points = []
    path_list = gruneisen_dict['path']
    for path_element in path_list:
        q_list = path_element['phonon']
        for q_element in q_list:
            q = q_element['q-position']
            points.append(q)
    qpoints = np.array(points)
    
    return qpoints

def get_parameters(name_parameter):
    """Finds the Gruneisen parameters and the equilibrium frequencies"""
    modes = []
    for i in range(n_modes):
        points = []
        path_list = gruneisen_dict['path']
        for path_element in path_list:
            q_list = path_element['phonon']
            for q_element in q_list:
                data_list = q_element['band']
                for j,parameter in enumerate(data_list):
                    if i == j:
                        points.append(parameter[name_parameter])
        modes.append(points)
    modes_np = np.array(modes)
    
    return modes_np

def dwdV():
    """Calculates dw/dV from the Gruneisen parameters, the equilibrium frequency, and the equilibrium volume"""
    dwdV = -(get_parameters('gruneisen') * get_parameters('frequency')) / get_V0()
    
    return dwdV

def w_T():
    """Calculates the temperature dependent frequencies"""
    temperatures = []
    zero = get_parameters('frequency')
    temperatures.append(zero)
    for i in range(1, len(get_volumes_T())):
        delta_V = get_volumes_T()[i] - get_V0()
        w_T = delta_V * dwdV() + get_parameters('frequency')
        temperatures.append(w_T)
        
    return temperatures

temperatures = []
for T in inclusive_range(Tmin, Tmax, Tstep):
    temperatures.append(T)
for i in range(len(temperatures)):
    phonon_disp = PhononBandStructureSymmLine(get_q(), w_T()[i], get_reciprocal_vectors(), labels_dict = labels).as_dict()
    filename = str(temperatures[i]) + 'K'
    with open(filename, 'w') as outfile:
        json.dump(phonon_disp, outfile)